# Ryanair Datathon Report

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
sns.__version__

'0.12.0'

In [49]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import sklearn.datasets as datasets

# Data Visualization

##### For this case, we were given two datasets. The first dataset called "train" is limited to basic information and it contains the same features as the "test" dataset. The second dataset is "train_extended" which we will use for the visualisation, as it allows us to detect potential outliers, and brainstorm on new features that we could potentially add to the basic "train"  and "test" datasets and that would help us in predicting the model.

In [50]:
# We import all the datasets

In [51]:
df_train=pd.read_csv("train.csv")
df=pd.read_csv("train_extended.csv")
df2=pd.read_csv("test.csv")
dfs=[df_train,df,df2]

In [52]:
from pandas_profiling import ProfileReport

In [53]:
profile = ProfileReport(df, title="Pandas Profiling Report")

We realize that among the variables that are included both in the basic "train" dataset and the "trainextended" dataset, we have 6 categorical variables, two timeseries variables, and 7 continuous variables including the target variable "TeledyneRampWeight". It seems like "AircraftRegistration" values are well balanced, however the variables "AircraftCapacity", "AircraftTypegroup","AOC description", "Carrier" and "Service Description" are unbalanced.

In [54]:
import seaborn as sns
from scipy.stats import norm, skew
from scipy import stats
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn 
pd.set_option('display.max_columns', None)
sns.__version__

'0.12.0'

In [55]:
num_feat = ['BlockTimeScheduled','Adults','Children', 'Freight', 'Infants', 'Bags', 'TeledyneRampWeight']
feat_skew = df[num_feat].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' : feat_skew})
skewness

,Skew
Freight,872.378222
TeledyneRampWeight,4.539957
Infants,2.561255
Children,2.547329
Bags,2.003505
BlockTimeScheduled,0.777010
Adults,-0.812129



We could detect outliers by comparing values in the dataset with real life possibilities, for example the planes in our dataset 

cannot exceed 85000kg, and we will actaly proceed to doing that. However the plots give us also valuable information. We can 

start with freight and detect the presence of outliers, looking at its histogram boxplot and qqplot, and considering that its 

skewness is of a value 872.3799316 with a mean of 40k a minimum of 0 and a maximum of 22233333. Moving on we can notice the 

presence of outliers for infants, children and bags. However the distribution of these variables is right skewed, this is 

actually understandable so this is to be taken into account when performing outliers removal. The same applies for the adults 

variable even thoguh the distribution is left skewed. The variable BlockTimeScheduled seems to be slightly skewed however we 

will be able to check its validity from a first a approach just by validating whether it is equal or not to the substraction of 

the Departure and Ariival Scheduled. Finally, the target variable seems tp have extreme unusual values as we can see in the 

qqplot and boxplot starting from values 100k.

# Data Cleaning

Looking at the range of the values for each variable and the sensitivity of each variable, we made the assumption that a regular outlier technique would not well clean the dataset. We validated this assumption through using zscore, IQR, IsolationForest, and Local Outlier Factor. These techniques would clean too much data or they would need to be manually customized for each variable. Therefore we proceeded cleaning the variables as follows:

##### We started by only selecting the rows in the dataset where the BlockTimeScheduled was higher than the flight time as the opposite is impossible.

In [56]:
df=df[df["FlightTime"]<df["BlockTimeScheduled"]]

In [57]:
len(df)

753819

##### Then we selected only the rows where the  TeledyneRampWeight is for sure higher than the basic empty weight of each specific aircraft as the opposite is impossible

In [58]:
dfa=df[(df['AircraftTypeGroup']=="NG") & (df['TeledyneRampWeight']<=41140)].index
df=df.drop(dfa)
dfb=df[(df['AircraftTypeGroup']=="Airbus") & (df['TeledyneRampWeight']<=42600)].index
df=df.drop(dfb)
dfc=df[(df['AircraftTypeGroup']=="Max") & (df['TeledyneRampWeight']<=41140)].index
df=df.drop(dfc)

In [59]:
len(df)

753703

##### In order to remove reight outliers, we selected only the rows where the basic empty weight of each particular airplane + freight would not  exceed the aorcraft associated maximum take off weight

In [60]:
dfe= df[(df['AircraftTypeGroup']=="Max") & (df['Freight']+45070>82191)].index 
df=df.drop(dfe)
dfg= df[(df['AircraftTypeGroup']=="NG") & (df['Freight']+41140>79000)].index 
df=df.drop(dfg)
dff= df[(df['AircraftTypeGroup']=="Airbus") &(df['Freight']+42060>77000)].index 
df=df.drop(dff)

In [61]:
len(df)

753692

##### We remove the rows where the ScheduledRoute is different from the Actual route because this behaviour could be due to many reasonsthat we cannot know. Weather, Lack of fuel, overweight, a combination of all... Also because we tried removing them and it performed better. We chose to delete them.

In [62]:
dfi=df[df["ScheduledRoute"]!=df["ActualRoute"]].index 
df=df.drop(dfi)

In [63]:
len(df)

753466

##### We deleted the rows where the ramp weight is higher than the planned  takeoffweight because practically the ramp weight should be lower since the fuel weight used for the taxiout is deduced from the ramp weight. After trial, removing these rows performed better. 

In [64]:
df=df[~(df["TeledyneRampWeight"]>(df["PlannedTOW"]))]

##### The rampweight should always be higher than the plannedzero fuel weight considering that the fuelweight is added. However it could happen that the prediction of the zero fuel weight is biased, therefore we put a margin. After trying, dropping these rows with the associated margin performed better.

In [65]:
dfe= df[(df['AircraftTypeGroup']=="Max") & (df["TeledyneRampWeight"]<df["PlannedZeroFuelWeight"]-((df["PlannedZeroFuelWeight"]-45070)*10/100))].index 
df=df.drop(dfe)
dfg= df[(df['AircraftTypeGroup']=="NG") & (df["TeledyneRampWeight"]<df["PlannedZeroFuelWeight"]-((df["PlannedZeroFuelWeight"]-41140)*10/100))].index 
df=df.drop(dfg)
dff= df[(df['AircraftTypeGroup']=="Airbus") & (df['AircraftCapacity']==180) & (df["TeledyneRampWeight"]<df["PlannedZeroFuelWeight"]-((df["PlannedZeroFuelWeight"]-42060)*10/100))].index 
df=df.drop(dff)

In [66]:
len(df)

618990

# Feature Engineering

In [67]:
test=df['ScheduledRoute'].str.split('-',n = 1, expand = True)
df["DepartureLocation"]= test[0]
df["ArrivalLocation"]= test[1]

In [68]:
airports=pd.read_csv("GlobalAirportDatabase.txt", sep=",")
airports['longitude']=airports['longitude'].astype(float)
airports['latitude']=airports['latitude'].astype(float)

for departure in df['DepartureLocation'].unique():
    departure_df= df[df['DepartureLocation']==departure]
    val= airports[airports['iata']==departure].iloc[:, [-2,-1]].values[0]
    departure_df['D_long']=val[1]
    departure_df['D_lat']=val[0]
    df.loc[df['DepartureLocation']==departure, 'D_long']= val[1]
    df.loc[df['DepartureLocation']==departure, 'D_lat']= val[0]

    
for Arrival in df['ArrivalLocation'].unique():
    Arrival_df2= df[df['ArrivalLocation']==Arrival]
    val= airports[airports['iata']==Arrival].iloc[:, [-2,-1]].values[0]
    Arrival_df2['A_long']=val[1]
    Arrival_df2['A_lat']=val[0]
    df.loc[df['ArrivalLocation']==Arrival, 'A_long']= val[1]
    df.loc[df['ArrivalLocation']==Arrival, 'A_lat']= val[0]

R=6371000  
lon1,lat1=df['D_long'], df['D_lat']
lon2,lat2=df['A_long'],  df['A_lat']                          # radius of Earth in meters
phi_1=np.radians(lat1)
phi_2=np.radians(lat2)

delta_phi=np.radians(lat2-lat1)
delta_lambda=np.radians(lon2-lon1)

a=np.sin(delta_phi/2.0)**2+\
np.cos(phi_1)*np.cos(phi_2)*\
np.sin(delta_lambda/2.0)**2
c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))

meters=R*c                         # output distance in meters
km=meters/1000.0              # output distance in kilometers
miles=meters*0.000621371      # output distance in miles
feet=miles*5280
df['distance']= km 

In [69]:
df=df.drop(columns=["D_long","D_lat","A_long","A_lat"])

In [70]:
df=pd.get_dummies(df, columns=['ServiceDescription'])

df=pd.get_dummies(df, columns=['AircraftTypeGroup'])

df=pd.get_dummies(df, columns=['AOCDescription'])

df=pd.get_dummies(df, columns=['Carrier'])

df=pd.get_dummies(df, columns=['AircraftCapacity'])

df=pd.get_dummies(df, columns=['AircraftRegistration'])

df=pd.get_dummies(df, columns=['DepartureLocation'])

df=pd.get_dummies(df, columns=['ArrivalLocation'])

In [71]:
df=df.drop(columns=["ScheduledRoute"])

In [72]:
df["DepartureScheduled"]=pd.to_datetime(df["DepartureScheduled"])
df["ArrivalScheduled"]=pd.to_datetime(df["ArrivalScheduled"])
df["DepartureScheduled"]=pd.to_datetime(df['DepartureScheduled'],format='%y/%m/%d %H:%M:%S')
df["ArrivalScheduled"]=pd.to_datetime(df['ArrivalScheduled'],format='%y/%m/%d %H:%M:%S')
df['day_of_week'] = df['DepartureScheduled'].dt.day_name()
df['hour_of_day'] = df['DepartureScheduled'].dt.hour
df['month_of_year'] =df['DepartureScheduled'].dt.month
df=df.drop(columns=["DepartureScheduled","ArrivalScheduled"])

In [73]:
df=pd.get_dummies(df, columns=['month_of_year','hour_of_day','day_of_week'])

In [74]:
df=df.drop(columns={'PlannedTOW', 'PlannedZeroFuelWeight','TaxiOut','PlannedTripTime','FlightID','FlightNumber','ActualRoute',
'DepartureActual','ArrivalActual','BlockTime','TaxiOut','Burnoff','FlightTime'})

# Feature engineering for the test dataset

##### Because some of the values in the test dataset variables are not present in the train dataset we will need to apply the feature engineering to the test dataset and make the columns in both datasets match

In [75]:
df2=df2[['AircraftRegistration', 'AircraftCapacity',
       'AircraftTypeGroup', 'ServiceDescription', 'Carrier', 'AOCDescription',
       'ScheduledRoute', 'DepartureScheduled', 'ArrivalScheduled',
       'BlockTimeScheduled', 'Adults', 'Children', 'Freight', 'Infants',
       'Bags']]

In [76]:
test=df2['ScheduledRoute'].str.split('-',n = 1, expand = True)
df2["DepartureLocation"]= test[0]
df2["ArrivalLocation"]= test[1]

In [77]:
airports=pd.read_csv("GlobalAirportDatabase.txt", sep=",")
airports['longitude']=airports['longitude'].astype(float)
airports['latitude']=airports['latitude'].astype(float)

for departure in df2['DepartureLocation'].unique():
    departure_df= df2[df2['DepartureLocation']==departure]
    val= airports[airports['iata']==departure].iloc[:, [-2,-1]].values[0]
    departure_df['D_long']=val[1]
    departure_df['D_lat']=val[0]
    df2.loc[df2['DepartureLocation']==departure, 'D_long']= val[1]
    df2.loc[df2['DepartureLocation']==departure, 'D_lat']= val[0]

    
for Arrival in df2['ArrivalLocation'].unique():
    Arrival_df2= df2[df2['ArrivalLocation']==Arrival]
    val= airports[airports['iata']==Arrival].iloc[:, [-2,-1]].values[0]
    Arrival_df2['A_long']=val[1]
    Arrival_df2['A_lat']=val[0]
    df2.loc[df2['ArrivalLocation']==Arrival, 'A_long']= val[1]
    df2.loc[df2['ArrivalLocation']==Arrival, 'A_lat']= val[0]

R=6371000  
lon1,lat1=df2['D_long'], df2['D_lat']
lon2,lat2=df2['A_long'],  df2['A_lat']                          # radius of Earth in meters
phi_1=np.radians(lat1)
phi_2=np.radians(lat2)

delta_phi=np.radians(lat2-lat1)
delta_lambda=np.radians(lon2-lon1)

a=np.sin(delta_phi/2.0)**2+\
np.cos(phi_1)*np.cos(phi_2)*\
np.sin(delta_lambda/2.0)**2
c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))

meters=R*c                         # output distance in meters
km=meters/1000.0              # output distance in kilometers
miles=meters*0.000621371      # output distance in miles
feet=miles*5280
df2['distance']= km 

In [78]:
df2=pd.get_dummies(df2, columns=['ServiceDescription'])
df2=pd.get_dummies(df2, columns=['AircraftTypeGroup'])
df2=pd.get_dummies(df2,columns=['AOCDescription'])
df2=pd.get_dummies(df2,columns=['Carrier'])
df2=pd.get_dummies(df2,columns=['AircraftCapacity'])
df2=pd.get_dummies(df2,columns=['DepartureLocation'])
df2=pd.get_dummies(df2,columns=['ArrivalLocation'])
df2=pd.get_dummies(df2,columns=["AircraftRegistration"])
df2=df2.drop(columns=["ScheduledRoute"])

In [79]:
df2["DepartureScheduled"]=pd.to_datetime(df2["DepartureScheduled"])
df2["ArrivalScheduled"]=pd.to_datetime(df2["ArrivalScheduled"])
df2["DepartureScheduled"]=pd.to_datetime(df2['DepartureScheduled'],format='%y/%m/%d %H:%M:%S')
df2['day_of_week'] = df2['DepartureScheduled'].dt.day_name()
df2['hour_of_day'] = df2['DepartureScheduled'].dt.hour
df2['month_of_year'] =df2['DepartureScheduled'].dt.month

In [80]:
df2=df2.drop(columns={'DepartureScheduled', 'ArrivalScheduled'})

In [81]:
df2=pd.get_dummies(df2, columns=['month_of_year','hour_of_day','day_of_week'])

In [82]:
df2=df2.drop(columns=["D_long","D_lat","A_long","A_lat"])

In [83]:
x=df.drop(columns=["TeledyneRampWeight"])

In [84]:
for i in x.columns:
    if i not in df2.columns:
        print(i)

AircraftRegistration_EIFOH
DepartureLocation_LLA
DepartureLocation_SPC
DepartureLocation_SPU
ArrivalLocation_LYS
ArrivalLocation_VBY


In [85]:
for i in df2.columns:
    if i not in x.columns:
        print(i)

DepartureLocation_VBY
ArrivalLocation_PRN


In [86]:
for i in x.columns:
    if i not in df2.columns:
        df2[i]=0

In [87]:
for i in df2.columns:
    if i not in x.columns:
         x[i]=0

In [88]:
x = x.reindex(sorted(x.columns), axis=1)

In [89]:
df2 = df2.reindex(sorted(x.columns), axis=1)

In [90]:
x["TeledyneRampWeight"]=df["TeledyneRampWeight"]

In [91]:
df=x

In [92]:
df.shape

(618990, 1282)

In [93]:
df2.shape

(191309, 1281)

# Neural Network Architecture

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from sklearn import preprocessing

In [ ]:
# THIS MODEL GAVE A MIN SCORE ON RTT OF 591 AND ON KAGGLE 935
def create_model():
    model = Sequential([
        layers.Input(shape=(1282)),
        layers.Dense(642, activation='relu'),
        layers.Dense(321, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', 
                       loss='mean_absolute_error', 
                       metrics=[tf.keras.metrics.MeanAbsoluteError()]
                      )
    return model
model=create_model()

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint("N3/Mod1/NN3model_{epoch:02d}_{val_loss:.2f}", 
                                                      monitor = "val_mean_absolute_error",
                                                      save_weights_only=True)
callbacks = [ checkpointer]

In [ ]:
X = df.drop(columns=["TeledyneRampWeight"])
y = df["TeledyneRampWeight"]

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Model was trained for 200 epochs and we chose the epoch 120 to make the predictions
history = model.fit(x=X_train.values, y=Y_train.values, 
              epochs=200, verbose=2, callbacks=callbacks,
              validation_data = (X_test.values, y_test.values))

In [ ]:
model.load_weights(tf.train.latest_checkpoint('N3/Mod2'))
outputs_predicted = model.predict(rt)
outputs_predicted.shape
from sklearn import metrics
print('Mean Squared Error:', metrics.mean_squared_error(rtt["TeledyneRampWeight"], outputs_predicted))

In [ ]:
outputs_predicted = model.predict(df2)
df_submission_file = pd.read_csv("test.csv")
df_submission_file["TeledyneRampWeight"]=outputs_predicted
df_submission_file =df_submission_file[["FlightID", "TeledyneRampWeight"]]
df_submission_file=df_submission_file.set_index("FlightID")
df_submission_file
df_submission_file.to_csv("predictions.csv")